In [55]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
cur_path = "/home/agent_t/MLM_project/new_res/"
log_path = "/home/agent_t/MLM_project/logs/"

In [53]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertModel, BertForMaskedLM
import torch 
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

model = BertForMaskedLM.from_pretrained("bert-base-uncased")

split_num = 0.7 
torch.manual_seed(1)
data_file_path = "/home/agent_t/MLM_project/data/cori_19/combine_cor19.txt" 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

dataset = LineByLineTextDataset(tokenizer=tokenizer, block_size=512, file_path=data_file_path)

indices = torch.randperm(len(dataset)).tolist()
train_dataset = torch.utils.data.Subset(dataset, indices[:int(split_num * len(indices))]) 
test_dataset = torch.utils.data.Subset(dataset, indices[int(split_num * len(indices)):]) 


training_args = TrainingArguments(
    output_dir=cur_path,          # output directory
    num_train_epochs=50,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=log_path,            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,               # training dataset
    data_collator=data_collator
    eval_dataset=test_dataset            # evaluation dataset
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
#dataset[0]
trainer.train()

KeyboardInterrupt: 

In [19]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.cluster import KMeans
import json 

num_clusters = 5
clu0 = []
clu1 = []
clu2 = []
clu3 = []
clu4 = []

model = BertModel.from_pretrained("bert-base-uncased")
km_cluster_token = KMeans(n_clusters=num_clusters, max_iter=300, n_init=40, init='k-means++', n_jobs=-1)
km_cluster_sentence = KMeans(n_clusters=num_clusters, max_iter=300, n_init=40, init='k-means++', n_jobs=-1)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
with open("/home/agent_t/MLM_project/data/cori_19/combine_cor19.txt","r") as f:
    a = f.read()

normal_sentences = a.split("\n")

In [3]:
sen_list = []
for i in range(len(normal_sentences)):
    if len(normal_sentences[i]) == 0:
        continue 
    sen_list.append(normal_sentences[i])


In [4]:
inputs = tokenizer(sen_list, return_tensors="pt", padding=True)

In [5]:
inputs["attention_mask"].size()

torch.Size([70278, 424])

In [6]:
# setting buffer 
# sentence level 
mod = len(sen_list) // 100
sent_total = []
for i in range(10):
    tmp = {}
    tmp["input_ids"] = inputs["input_ids"][i*100:(i+1)*100]
    tmp["token_type_ids"] = inputs["token_type_ids"][i*100:(i+1)*100]
    tmp["attention_mask"] = inputs["attention_mask"][i*100:(i+1)*100]
    sentence_outputs = model(**tmp)[1].detach().numpy()
    sent_total.append(sentence_outputs)
    
   # km_cluster_sentence.fit(sentence_outputs)
    #print(km_cluster_sentence.labels_)

In [ ]:
import h5py 
f = h5py.file("bert_sentence_dim.hdf5", "w")
d1 = f.create_dataset("dataset1", (), "float")

In [7]:
print(len(sen_list) // 100)
len(sent_total)

702


10

In [26]:
import numpy as np

init = sent_total[0]
for i in range(1, len(sent_total)):
    init = np.concatenate((init, sent_total[i]), axis=0)

km_cluster_sentence.fit(init)

init_sent = sen_list#[:1001]
init_label = km_cluster_sentence.labels_


1001

In [27]:
for i in range(len(init_label)):
    if init_label[i] == 0:
        clu0.append(init_sent[i])
    elif init_label[i] == 1:
        clu1.append(init_sent[i])
    elif init_label[i] == 2:
        clu2.append(init_sent[i])
    elif init_label[i] == 3:
        clu3.append(init_sent[i])
    elif init_label[i] == 4:
        clu4.append(init_sent[i])


In [28]:
clu0

['Are schools closing for the rest of the year in Winston Salem',
 "I'm predicting a large increase in cases due to the following 2 observations:",
 '\\-Social gatherings less than 10 people... two of my neighbors had drunken blow outs this weekend with lots of booze and  waaay more than 10 people plus the grocery stores were packed',
 "As a retail worker near Wake, that's my biggest fear. We're doing extra cleaning during and after store hours to be on top of this. Hopefully it makes a difference!",
 'Highly likely.    My wife is in the medical field and indicated this to me at least a week ago',
 '^(I) ^(AM) ^(A) ^(BOT) ^(|) ^(REPLY) ^(**!IGNORE**) ^(AND) ^(I) ^(WILL) ^(STOP) ^(REPLYING) ^(TO) ^(YOUR) ^(COMMENTS)',
 'COVID-19 case confirmed at the Hayward Industries plant in Clemmons',
 'I think the official order has specific industries which are deemed essential listed— your company may not fall under those guidelines.',
 'I second this, a few days/weeks at home is worth it in the 

In [29]:
clu1

["I think is important to say that Duke energy won't be following up on those who've had disconnection notices",
 ' Central Terrace United Methodist Church – New Story ',
 ' Children’s Law Center of Central North Carolina ',
 'If you’re a conservative you’ll get downvoted for posting any facts on here so don’t do it!',
 'Would be nicer if they didn’t just lay off ALL of their contract workers on Friday with no advance notice.',
 'they are online classes!  the picture is from when we had normal class',
 'Yo this was super helpful thank you',
 "That's good to know. That place is massive.",
 '**MAJOR UPDATE(S): [Stay At Home Order to begin Friday, March 27 at 5PM](https://www.cityofws.org/2680/Stay-At-Home-Order-Provisions); Statewide Monday, March 30 at 5PM**',
 '**Sources for information related to Coronavirus (COVID-19)**',
 '[City of Winston-Salem COVID-19 website](https://www.cityofws.org/2673/COVID-19-Coronavirus-Information) *edited 3/16/20 for direct link*',
 "If there are any spe

In [30]:
clu2

['**MAJOR UPDATE(S): [Stay At Home Order to begin Friday, March 27 at 5PM](https://www.cityofws.org/2680/Stay-At-Home-Order-Provisions); Statewide Monday, March 30 at 5PM**',
 '**Sources for information related to Coronavirus (COVID-19)**',
 '[City of Winston-Salem COVID-19 website](https://www.cityofws.org/2673/COVID-19-Coronavirus-Information) *edited 3/16/20 for direct link*',
 "If there are any specific closings or cancellations you know of, or if you need assistance with childcare, or if you just need some toilet paper, feel free to post in this thread. Hopefully this pandemic doesn't last too long, but that remains to be seen. I don't think any one of us can say we've experienced a situation quite like this in our lifetimes, but we'll pull through somehow.",
 'I am deeply concerned for our community. I do not think this city understands just how inter-connected we are to the rest of the world. I work at an "essential business"...but not the kind that puts food on our table. I als

In [ ]:
# setting buffer 
"""
please do not do vocab level 
"""
# vocab level:
# more important: need remove pad token 
# here, we should our input as one [num, 768], num means, how many vocab or sentence

token_outputs = model(**inputs)[0].squeeze(0).detach().numpy()

# del start and end, only for token level 
token_outputs = token_outputs[1:-1]
km_cluster_token.fit(token_outputs)
label_vocab = km_cluster_token.labels_  # 获取聚类标签
center_array = km_cluster_token.cluster_centers_  # 获取聚类中心
